In [1]:
from util.util import table_to_dataframe, get_table_names
import pandas as pd

In [2]:
databaseB = "A20240116"
password = '93150lbm!!'
tables = get_table_names('127.0.0.1', 'root', password, databaseB)

code = tables[0][0]
print(code)

005860


In [3]:
df = table_to_dataframe('127.0.0.1', 'root', password, databaseB, code)
df['체결시간'] = pd.to_datetime(df['체결시간'])
df = df.sort_values(by='체결시간', ignore_index=True)
df['time_diff'] = df['체결시간'].diff().dt.total_seconds()
df['현재가'] = pd.to_numeric(df['현재가'].astype(str).str.replace('+','').str.replace('-',''), errors='coerce')
df = df.dropna()



time_val = df['time_diff'].values
chaegul_data = df[['현재가','거래량']].values

chaegul_data

c:\Users\pari0\Desktop\simulator_test\util\util.py:53: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


array([[4640, '-16'],
       [4640, '-30'],
       [4640, '-50'],
       ...,
       [4615, '+1'],
       [4615, '+1'],
       [4615, '+1']], dtype=object)

In [4]:
price,amount =chaegul_data[0]

print(price,amount)

4640 -16


In [5]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QTableWidget, QTableWidgetItem, QVBoxLayout, QPushButton
from PyQt5.QtGui import QColor
from PyQt5.QtWidgets import (QApplication, QWidget
, QLineEdit, QTextBrowser, QPushButton, QVBoxLayout)
from PyQt5.QtWidgets import QApplication, QWidget, QTableWidget, QTableWidgetItem, QVBoxLayout, QProgressBar
from PyQt5.QtCore import QTimer

class MyApp(QWidget):

    def __init__(self, time, chaegul_data):
        super().__init__()
        self.chaegul_data = chaegul_data
        self.time = time 
        self.current_step = 0 
        self.initUI()

    def initUI(self):
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.append_text)
        self.startNextTimer()

        self.table = QTableWidget(0, 2)  # 0 rows and 2 columns
        self.table.setHorizontalHeaderLabels(['Price', 'Amount'])

        self.clear_btn = QPushButton('Clear')
        self.clear_btn.pressed.connect(self.clear_table)

        vbox = QVBoxLayout()
        vbox.addWidget(self.table, 0)
        vbox.addWidget(self.clear_btn, 1)

        self.setLayout(vbox)

        self.setWindowTitle('Dynamic QTableWidget')
        self.setGeometry(300, 300, 300, 300)
        self.show()

    def append_text(self):
        price, amount = self.chaegul_data[self.current_step]
        
        # Insert a new row at the beginning
        self.table.insertRow(0)

        # Create QTableWidgetItem for price and amount
        price_item = QTableWidgetItem(str(price))
        amount_item = QTableWidgetItem(amount)

        # Set color based on condition
        if '+' in amount:
            color = QColor('red')
        elif '-' in amount:
            color = QColor('blue')
        
        # color_black = QColor('black')

        price_item.setForeground(color)
        amount_item.setForeground(color)

        # Add items to the table
        self.table.setItem(0, 0, price_item)
        self.table.setItem(0, 1, amount_item)

        self.current_step += 1
        self.startNextTimer()

    def startNextTimer(self):
        if self.current_step < len(self.time):
            interval = int(self.time[self.current_step] * 1000)
            self.timer.start(interval)
        else:
            self.timer.stop()

    def clear_table(self):
        self.table.setRowCount(0)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp(time_val, chaegul_data)
    sys.exit(app.exec_())


SystemExit: 0

c:\Users\pari0\anaconda3\envs\simulator\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
## Ex 5-19. QTextBrowser.

import sys
from PyQt5.QtWidgets import (QApplication, QWidget
, QLineEdit, QTextBrowser, QPushButton, QVBoxLayout)
from PyQt5.QtWidgets import QApplication, QWidget, QTableWidget, QTableWidgetItem, QVBoxLayout, QProgressBar
from PyQt5.QtCore import QTimer


class MyApp(QWidget):

    def __init__(self,time, chaegul_data):
        super().__init__()
        self.chaegul_data = chaegul_data
        self.time = time 
        self.current_step = 0 
        self.text_lines = [] 
        self.initUI()

    def initUI(self):
        # self.le = QLineEdit()
        # self.le.returnPressed.connect(self.append_text)
        
        self.timer = QTimer(self)
        self.timer.timeout.connect(self.append_text)
        self.startNextTimer()

        self.tb = QTextBrowser()
        self.tb.setAcceptRichText(True)
        self.tb.setOpenExternalLinks(True)

        self.clear_btn = QPushButton('Clear')
        self.clear_btn.pressed.connect(self.clear_text)

        vbox = QVBoxLayout()
        # vbox.addWidget(self.le, 0)
        vbox.addWidget(self.tb, 0)
        vbox.addWidget(self.clear_btn, 1)

        self.setLayout(vbox)

        self.setWindowTitle('QTextBrowser')
        self.setGeometry(300, 300, 300, 300)
        self.show()

    def append_text(self):
        # new_text = self.le.text()
        price, amount = self.chaegul_data[self.current_step]
        new_text = str(price) + ' / ' + amount
        
        if '+' in amount : 
            color = 'red'
        elif '-' in amount : 
            color = 'blue'
        # colored_text = f'<span style="color:{color};">{new_text}</span>'
        colored_text = f'<span style="color:{color};">{new_text}</span>'
        self.text_lines.append(colored_text)
        
        
        reversed_html_text = '<br>'.join(reversed(self.text_lines))
        
        # Set the HTML content in QTextBrowser
        self.tb.setHtml(reversed_html_text)
        
        # current_text = self.tb.toPlainText()
        # combined_text = new_text + '\n' + current_text  
        # self.tb.setPlainText(combined_text)
        
        # self.tb.insertHtml(colored_text + '<br>' + current_text) 
        self.current_step += 1
        
    def startNextTimer(self):
        if self.current_step < len(self.time):
            interval = int(self.time[self.current_step] * 10)
            self.timer.start(interval)
        else:
            self.timer.stop() 



    def clear_text(self):
        self.tb.clear()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = MyApp(time_val,chaegul_data)
    sys.exit(app.exec_())


KeyboardInterrupt: 

SystemExit: 0

c:\Users\pari0\anaconda3\envs\simulator\lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
